# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../WeatherPy/city_weather_output.csv", index_col=0)
weather_df.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,fortuna,40.5982,-124.1573,282.04,100,90,1.25,US,1618733958
1,hasaki,35.7333,140.8333,291.15,27,20,2.57,JP,1618733741
2,ushuaia,-54.8000,-68.3000,284.15,54,75,4.12,AR,1618733959
3,albany,42.6001,-73.9662,279.82,68,97,0.89,US,1618733853
4,adiake,5.2863,-3.3040,300.15,83,20,1.03,CI,1618733892


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Store 'Lat' & 'Lng' into locations
locations = weather_df[['Lat', 'Lng']].astype(float)
humidity = weather_df['Humidity'].astype(float)

In [5]:
#Create heatmap
fig = gmaps.figure(zoom_level=1.5, center=(20,0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Convert Temperature from Kelvin to Fahrenheit
weather_df['Max Temp'] = 9/5 * (weather_df['Max Temp'] - 273) + 32
#Convert Wind Speed from M/S to MPH
weather_df['Wind Speed'] = weather_df['Wind Speed'] * 2.237

In [7]:
#ideal_temp = < 80 & > 70
#ideal_wind = 10
#ideal_cloud = 10
ideal_df = weather_df.loc[(weather_df['Max Temp'] <= 80) & (weather_df['Max Temp'] > 70) &
                          (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] < 10), :]

#Drop any rows with null values
ideal_df = ideal_df.dropna()
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,rikitea,-23.1203,-134.9692,77.450,71,5,5.65961,PF,1618733960
23,busselton,-33.6500,115.3333,76.262,45,0,4.00423,AU,1618733521
27,adrar,20.5022,-10.0711,72.716,25,0,7.31499,MR,1618733522
34,high rock,26.6208,-78.2833,75.470,83,0,4.60822,BS,1618733525
54,makakilo city,21.3469,-158.0858,73.670,73,1,5.74909,US,1618733752
77,east london,-33.0153,27.9116,79.070,34,0,2.30411,ZA,1618733542
116,alofi,-19.0595,-169.9187,71.870,94,3,1.14087,NU,1618733558
126,oussouye,12.4850,-16.5469,71.870,94,0,2.30411,SN,1618733561
128,esperance,-33.8667,121.9000,77.270,31,0,8.05320,AU,1618733562
146,port elizabeth,-33.9180,25.5701,79.070,36,0,3.44498,ZA,1618733528


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_df.copy()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,rikitea,-23.1203,-134.9692,77.450,71,5,5.65961,PF,1618733960
23,busselton,-33.6500,115.3333,76.262,45,0,4.00423,AU,1618733521
27,adrar,20.5022,-10.0711,72.716,25,0,7.31499,MR,1618733522
34,high rock,26.6208,-78.2833,75.470,83,0,4.60822,BS,1618733525
54,makakilo city,21.3469,-158.0858,73.670,73,1,5.74909,US,1618733752
77,east london,-33.0153,27.9116,79.070,34,0,2.30411,ZA,1618733542
116,alofi,-19.0595,-169.9187,71.870,94,3,1.14087,NU,1618733558
126,oussouye,12.4850,-16.5469,71.870,94,0,2.30411,SN,1618733561
128,esperance,-33.8667,121.9000,77.270,31,0,8.05320,AU,1618733562
146,port elizabeth,-33.9180,25.5701,79.070,36,0,3.44498,ZA,1618733528


In [13]:
#Find first hotel for each city located within 5000 meters
#Hotel Name, City, Country

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key":g_key    
}

for index, row in hotel_df.iterrows():
    
    #Get lat & lng
    location = f"{row['Lat']}, {row['Lng']}"
    
    #Changes the coordinates per row
    params['location'] = location
    
    #Assemble url
    response = requests.get(base_url, params = params).json()
    results = response['results']
    
    try:
        #Gets the name of the first result
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
        print(f"The closest hotel to {row['City']} is {hotel_df.loc[index, 'Hotel Name']}")
    
    except IndexError:
        print(f"No hotels near {row['City']}")


The closest hotel to rikitea is Pension Maro'i
The closest hotel to busselton is Observatory Guest House
No hotels near adrar
The closest hotel to high rock is Hideaway Bahamas Beach Club
The closest hotel to makakilo city is Marriott's Ko Olina Beach Club
The closest hotel to east london is Tu Casa
The closest hotel to alofi is Taloa Heights
The closest hotel to oussouye is Auberge du Routard
The closest hotel to esperance is Hospitality Esperance
The closest hotel to port elizabeth is 39 On Nile Guest House
The closest hotel to pochutla is Hotel Posada San Jose
The closest hotel to sao filipe is Tortuga B&B
The closest hotel to piedra gorda is Centro comunal Piedra Gorda
The closest hotel to acapulco is HS Hotsson Smart Acapulco
The closest hotel to vila velha is Quality Suites Vila Velha
The closest hotel to birjand is Birjand Tourist Hotel
The closest hotel to ewa beach is Uncle Junior Robert
The closest hotel to macia is Tuck shop my life dcoza
The closest hotel to ulundi is Ekhet

In [12]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,rikitea,-23.1203,-134.9692,77.450,71,5,5.65961,PF,1618733960,Pension Maro'i
23,busselton,-33.6500,115.3333,76.262,45,0,4.00423,AU,1618733521,Observatory Guest House
27,adrar,20.5022,-10.0711,72.716,25,0,7.31499,MR,1618733522,NaN
34,high rock,26.6208,-78.2833,75.470,83,0,4.60822,BS,1618733525,Hideaway Bahamas Beach Club
54,makakilo city,21.3469,-158.0858,73.670,73,1,5.74909,US,1618733752,Marriott's Ko Olina Beach Club
77,east london,-33.0153,27.9116,79.070,34,0,2.30411,ZA,1618733542,Tu Casa
116,alofi,-19.0595,-169.9187,71.870,94,3,1.14087,NU,1618733558,Taloa Heights
126,oussouye,12.4850,-16.5469,71.870,94,0,2.30411,SN,1618733561,Auberge du Routard
128,esperance,-33.8667,121.9000,77.270,31,0,8.05320,AU,1618733562,Hospitality Esperance
146,port elizabeth,-33.9180,25.5701,79.070,36,0,3.44498,ZA,1618733528,39 On Nile Guest House


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


# Display figure
